We want to test whether the neural network [chemprop](https://github.com/chemprop/chemprop) performs well at our classification task. 
Running this on a seperate conda instance that has chemprop installed. 

In [ ]:
!cd .. && export PYTHONPATH=$PWD && chemprop_train --quiet --extra_metrics accuracy --data_path data/processed/smiles_to_receptor_one_hot.csv --dataset_type classification --save_dir models/chemprop_test/ 

 Manually truncated.


Now that we have trained our neural net, let's see how it performed.

In [9]:
import pandas as pd

df = pd.read_csv("../models/chemprop_test/test_scores.csv")
df.head()

,Task,Mean auc,Standard deviation auc,Fold 0 auc,Mean accuracy,Standard deviation accuracy,Fold 0 accuracy
0,"11-beta-hydroxysteroid dehydrogenase 1, 11-bet...",0.998428,0.0,0.998428,0.999102,0.0,0.999102
1,5-lipoxygenase activating protein,0.999995,0.0,0.999995,0.999858,0.0,0.999858
2,"6-phosphofructo-2-kinase/fructose-2,6-bisphosp...",0.999979,0.0,0.999979,0.999953,0.0,0.999953
3,ALK tyrosine kinase receptor,0.999047,0.0,0.999047,0.998771,0.0,0.998771
4,"Acetyl-CoA carboxylase 2, Acetyl-CoA carboxyla...",0.999986,0.0,0.999986,0.999811,0.0,0.999811


In [10]:
df.describe()

,Mean auc,Standard deviation auc,Fold 0 auc,Mean accuracy,Standard deviation accuracy,Fold 0 accuracy
count,120.000000,120.0,120.000000,120.000000,120.0,120.000000
mean,0.996506,0.0,0.996506,0.997530,0.0,0.997530
std,0.004206,0.0,0.004206,0.003403,0.0,0.003403
min,0.971807,0.0,0.971807,0.982108,0.0,0.982108
25%,0.994863,0.0,0.994863,0.997246,0.0,0.997246
50%,0.997647,0.0,0.997647,0.998795,0.0,0.998795
75%,0.999528,0.0,0.999528,0.999551,0.0,0.999551
max,1.000000,0.0,1.000000,0.999976,0.0,0.999976


Mean accuracy of 0.9975, with a minimum accuracy of 0.9821. Looks like chemprop performs very well. 

In [12]:
!cd .. && echo $'smiles\nCC(=O)OC1=CC=CC=C1C(=O)O\nO=C1NC2=CC=C([N+]([O-])=O)C=C2C(C3=CC=CC=C3F)=NC1' > test.csv

In [13]:
!cd .. && export PYTHONPATH=$PWD && chemprop_predict --test_path test.csv --checkpoint_dir models/chemprop_test/ --preds_path test_preds.csv

/home/ian/anaconda3/envs/target-chemprop/lib/python3.8/site-packages/tap/tap.py:312: UserWarning: add_arguments is deprecated and will be removed on January 1st, 2021 please override "configure" instead and call add_argument there. 
  warn('add_arguments is deprecated and will be removed on January 1st, 2021'
Loading training args
Loading data
2it [00:00, 13273.11it/s]
100%|██████████████████████████████████████████| 2/2 [00:00<00:00, 70492.50it/s]
Validating SMILES
Test size = 2
Predicting with an ensemble of 1 models
  0%|                                                     | 0/1 [00:00<?, ?it/s]Loading pretrained parameter "encoder.encoder.0.cached_zero_vector".
Loading pretrained parameter "encoder.encoder.0.W_i.weight".
Loading pretrained parameter "encoder.encoder.0.W_h.weight".
Loading pretrained parameter "encoder.encoder.0.W_o.weight".
Loading pretrained parameter "encoder.encoder.0.W_o.bias".
Loading pretrained parameter "ffn.1.weight".
Loading pretrained parameter "ffn.1.bia

In [14]:
df = pd.read_csv("../test_preds.csv")

df.set_index("smiles", inplace=True)

for _, row in df.iterrows():
    print(row.nlargest(5))
    print()

Cyclooxygenase-2, Cyclooxygenase-1                                                                                                                                                                                                                                                                               0.612759
Carbonic anhydrase II, Carbonic anhydrase VA, Carbonic anhydrase IX, Carbonic anhydrase I, Carbonic anhydrase IV, Carbonic anhydrase VB, Carbonic anhydrase VI, Carbonic anhydrase VII, Carbonic anhydrase XII, Carbonic anhydrase XIV, Carbonic anhydrase III, Carbonic anhydrase XIII, Carbonic anhydrase 1    0.376249
Histamine H1 receptor, Histamine H2 receptor, Histamine H3 receptor, Histamine H4 receptor                                                                                                                                                                                                                       0.013905
Estrogen receptor alpha, Estrogen receptor beta           

As we can see it identifies Asprin as acting on COX, and the experimental GABAergic NPS from the Predict NPS test set has GABA as the fourth most likely receptor. Not bad performance.

In [15]:
!cd .. && rm test.csv test_preds.csv